In [7]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from sklearn.neighbors import KDTree
from caveclient import CAVEclient
from skeleton_plot import skel_io

# Step 1: Initialize CAVEclient and load neuron skeleton
client = CAVEclient(datastack_name='minnie65_public')
materialization_version = 1078  # Adjust as needed
client.materialize.version = materialization_version

# Replace with actual segment and nucleus IDs
segment_id = 864691135122603047
nucleus_id = 292685
skel_path = "s3://bossdb-open-data/iarpa_microns/minnie/minnie65/skeletons/v661/skeletons/"
skel_filename = f"{segment_id}_{nucleus_id}.swc"

# Load skeleton
skel = skel_io.read_skeleton(skel_path, skel_filename)

# Step 2: Build graph from skeleton
# Create a graph from the skeleton
G = nx.Graph()
node_positions = {i: skel.vertices[i] for i in range(skel.n_vertices)}

# Add nodes
for i, coord in node_positions.items():
    G.add_node(i)

# Add edges using Euclidean distances
for edge in skel.edges:
    src, tgt = edge
    coord1 = node_positions[src]
    coord2 = node_positions[tgt]
    weight = np.linalg.norm(coord1 - coord2)
    G.add_edge(src, tgt, weight=weight)

# Step 3: Compute Laplacian
L = nx.laplacian_matrix(G, weight='weight')

# Step 4: Compute eigenvectors
k = 20  # Number of modes
eigenvalues, eigenvectors = eigsh(L, k=k, which='SM')

# Step 5: Query synapse annotations
synapse_df = client.materialize.query_table('synapse_table', split_positions=True)
# Filter for synapses associated with the neuron
synapse_df = client.materialize.query_table(
    'synapse_table',
    split_positions=True,
    filter_in_dict={'post_pt_root_id': [segment_id]}
)

# Step 6: Map synapses to nearest skeleton nodes
skeleton_coords = np.array(list(node_positions.values()))
tree = KDTree(skeleton_coords)
synapse_coords = synapse_df[['x', 'y', 'z']].values
distances, indices = tree.query(synapse_coords, k=1)
nearest_nodes = [list(node_positions.keys())[idx[0]] for idx in indices]

# Step 7: Embed synapses
embeddings = []
for node_id in nearest_nodes:
    node_index = list(G.nodes).index(node_id)
    embedding = eigenvectors[node_index] / eigenvalues
    embeddings.append(embedding)

# Convert to DataFrame
embedding_df = pd.DataFrame(embeddings, columns=[f'mode_{i+1}' for i in range(k)])
embedding_df['synapse_id'] = synapse_df['id'].values

# Save embeddings
embedding_df.to_csv('synapse_embeddings.csv', index=False)


KeyboardInterrupt: 